In [1]:
import findspark

In [2]:
findspark.init('/home/vboxuser/spark-3.5.0-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('rating').getOrCreate()

In [7]:
from pyspark.ml.recommendation import ALS

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

In [21]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [22]:
import pandas as pd
data = pd.read_csv('/home/vboxuser/Downloads/ratings.csv', delimiter='\t')

In [24]:
spark_data = spark.createDataFrame(data)

In [25]:
spark_data.columns

['Unnamed: 0',
 'user_id',
 'movie_id',
 'rating',
 'timestamp',
 'user_emb_id',
 'movie_emb_id']

In [14]:
len(data)

TypeError: object of type 'DataFrame' has no len()

In [15]:
data.count()

1000209

In [26]:
spark_data.show()

24/01/11 12:31:11 WARN TaskSetManager: Stage 8 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------+--------+------+---------+-----------+------------+
|Unnamed: 0|user_id|movie_id|rating|timestamp|user_emb_id|movie_emb_id|
+----------+-------+--------+------+---------+-----------+------------+
|         0|      1|    1193|     5|978300760|          0|        1192|
|         1|      1|     661|     3|978302109|          0|         660|
|         2|      1|     914|     3|978301968|          0|         913|
|         3|      1|    3408|     4|978300275|          0|        3407|
|         4|      1|    2355|     5|978824291|          0|        2354|
|         5|      1|    1197|     3|978302268|          0|        1196|
|         6|      1|    1287|     5|978302039|          0|        1286|
|         7|      1|    2804|     5|978300719|          0|        2803|
|         8|      1|     594|     4|978302268|          0|         593|
|         9|      1|     919|     4|978301368|          0|         918|
|        10|      1|     595|     5|978824268|          0|      

In [28]:
spark_data.describe().show()

24/01/11 12:36:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/11 12:36:46 WARN TaskSetManager: Stage 9 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+------------------+------------------+------------------+--------------------+-----------------+------------------+
|summary|        Unnamed: 0|           user_id|          movie_id|            rating|           timestamp|      user_emb_id|      movie_emb_id|
+-------+------------------+------------------+------------------+------------------+--------------------+-----------------+------------------+
|  count|           1000209|           1000209|           1000209|           1000209|             1000209|          1000209|           1000209|
|   mean|          500104.0| 3024.512347919285|1865.5398981612843| 3.581564453029317| 9.722436954046655E8|3023.512347919285|1864.5398981612843|
| stddev|288735.61203547445|1728.4126949000054|1096.0406894572582|1.1171018453732544|1.2152558939912412E7|1728.412694900005|1096.0406894572582|
|    min|                 0|                 1|                 1|                 1|           956703932|                0|            

In [29]:
main_dataset,other_dataset = spark_data.randomSplit([0.1,0.9])

In [30]:
main_dataset.describe().show()

24/01/11 12:38:50 WARN TaskSetManager: Stage 12 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|summary|        Unnamed: 0|           user_id|          movie_id|            rating|           timestamp|       user_emb_id|      movie_emb_id|
+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|  count|            100411|            100411|            100411|            100411|              100411|            100411|            100411|
|   mean|500287.28449074307|3025.4521317385547|1864.7829520670045|3.5870073995876948| 9.722195204196851E8|3024.4521317385547|1863.7829520670045|
| stddev|288500.06560146867|1727.0310558667584| 1095.479942822121|1.1156178670634689|1.2111888165938595E7|1727.0310558667582| 1095.479942822121|
|    min|                 3|                 1|                 1|                 1|           956704219|                 0|     

In [34]:
train_data,test_data = main_dataset.randomSplit([0.7,0.3])

In [37]:
train_data.columns

['Unnamed: 0',
 'user_id',
 'movie_id',
 'rating',
 'timestamp',
 'user_emb_id',
 'movie_emb_id']

In [38]:
als = ALS(maxIter = 5, regParam=0.01, userCol='user_id', itemCol='movie_id', ratingCol='rating')

In [39]:
model = als.fit(train_data)

24/01/11 12:54:09 WARN TaskSetManager: Stage 15 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.
24/01/11 12:54:11 WARN TaskSetManager: Stage 16 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.
24/01/11 12:54:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/11 12:54:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [40]:
predictions = model.transform(test_data)

In [41]:
predictions.show()

24/01/11 12:55:20 WARN TaskSetManager: Stage 49 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------+--------+------+---------+-----------+------------+-----------+
|Unnamed: 0|user_id|movie_id|rating|timestamp|user_emb_id|movie_emb_id| prediction|
+----------+-------+--------+------+---------+-----------+------------+-----------+
|        22|      1|    1270|     5|978300055|          0|        1269|  3.6493287|
|        33|      1|     588|     4|978824268|          0|         587|  4.2211695|
|       671|      8|    2396|     5|978229524|          7|        2395|  3.8475852|
|       341|      5|    2725|     2|978246162|          4|        2724|    3.63661|
|         5|      1|    1197|     3|978302268|          0|        1196|  4.2042875|
|       286|      5|     581|     3|978244808|          4|         580|   8.951179|
|       774|      9|      16|     4|978226599|          8|          15|  4.3396406|
|       502|      6|    3699|     4|978236567|          5|        3698|        NaN|
|       521|      6|    1028|     4|978237767|          5|        1027|     

In [42]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol='rating', predictionCol = 'prediction')

In [43]:
rmse = evaluator.evaluate(predictions)

24/01/11 12:58:15 WARN TaskSetManager: Stage 100 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


In [44]:
print(rmse)

nan


In [51]:
single_user = test_data.filter(test_data['user_id']==8).select(['movie_id','user_id','rating'])

In [52]:
single_user.show()

24/01/11 13:02:12 WARN TaskSetManager: Stage 191 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.
24/01/11 13:02:13 WARN TaskSetManager: Stage 192 contains a task of very large size (6569 KiB). The maximum recommended task size is 1000 KiB.


+--------+-------+------+
|movie_id|user_id|rating|
+--------+-------+------+
|     650|      8|     5|
|    2291|      8|     5|
|    3259|      8|     4|
|    2396|      8|     5|
|    1673|      8|     5|
+--------+-------+------+



In [49]:
recommendation = model.transform(single_user)

In [53]:
recommendation.orderBy('prediction',ascending=False).show()

24/01/11 13:02:30 WARN TaskSetManager: Stage 193 contains a task of very large size (6367 KiB). The maximum recommended task size is 1000 KiB.


+--------+-------+----------+
|movie_id|user_id|prediction|
+--------+-------+----------+
|    2396|      8| 3.8475852|
|     650|      8| 3.1854467|
|    2291|      8| 2.7093437|
|    1673|      8| 2.2676625|
|    3259|      8| 1.6646817|
+--------+-------+----------+

